In [11]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
from tensorflow.keras import datasets, layers, models

In [12]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

In [13]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0] - 1

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0] - 1  

In [14]:
imgs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs.append(img)
print("--- %s seconds ---" % (time.time() - start_time))  

imgs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    imgs_test.append(img)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 77.71262311935425 seconds ---
--- 20.534557104110718 seconds ---


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(188, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(np.array(imgs), emotion_idx, epochs=10, validation_data=(np.array(imgs_test), test_emotion_idx))

Train on 1999 samples, validate on 501 samples
Epoch 1/10
1999/1999 [==============================] - 481s 241ms/step - loss: 3.1356 - acc: 0.0445 - val_loss: 3.0912 - val_acc: 0.0579
Epoch 2/10
1999/1999 [==============================] - 490s 245ms/step - loss: 3.0909 - acc: 0.0415 - val_loss: 3.0911 - val_acc: 0.0559
Epoch 3/10
 768/1999 [==========>...................] - ETA: 4:32 - loss: 3.0926 - acc: 0.0573

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-26618e42ffc2>", line 13, in <module>
    history = model.fit(np.array(imgs), emotion_idx, epochs=10, validation_data=(np.array(imgs_test), test_emotion_idx))
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1363, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_arrays.py", line 264, in fit_loop
    outs = f(ins_batch)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2914, in __call__
    fetched = self._callable_fn(*array_vals)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1382, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the abov

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 642, in _abort_queue
    ide